In [ ]:
import requests
import tiktoken
secret = "W9bcrkRD7Zmbv2fEjgG7kCLN1Wexbg"
client_id=  "V63aQL-O_zehkUdVfXLPkA"
# tutorial: https://www.youtube.com/watch?v=FdjVoOf9HN4

: 

### get access token

In [5]:
base_url = 'https://www.reddit.com/'
data = {'grant_type': 'password', 'username': 'lamboqin_stealth',
        'password': '159753Qw!'}
auth = requests.auth.HTTPBasicAuth(client_id, secret)
r = requests.post(base_url + 'api/v1/access_token',
                  data=data,
                  headers={'user-agent': 'APP-NAME by REDDIT-USERNAME'},
		  auth=auth)
d = r.json()
token = 'bearer ' + d['access_token']
headers = {'Authorization': token, 'User-Agent': 'APP-NAME by REDDIT-USERNAME'}

Make request to https://oauth.reddit.com

In [1]:
base_url = 'https://oauth.reddit.com'
response = requests.get(base_url + '/api/v1/me', headers=headers)

if response.status_code == 200:
    print(response.json()['name'], response.json()['comment_karma'])

NameError: name 'requests' is not defined

### functions

In [5]:
def print_str_as_blocks(str, char_limit=70, indent=0) -> str:
    line = ""
    output = ""
    for char in str:
        if char == "\n":
            output+=line+'\n'
            line = ""
            continue
                
        if len(line) > 70 and char == " ":
            output+=line+"\n"
            line = ""+"  "*indent
            continue
        line+=char
    if len(line) > 0:
        output+=line
        
    print(output)

def add_indent(str, indent:int):
    output = ""
    line = ""
    indent_str = "  "*indent
    for char in str:
        if char == "\n":
            output += indent_str+line+"\n"
            line = ""
        else:
            line+=char
    if len(line) > 0:
        output+=indent_str+line
    return output

def remove_extra_lines(str):
    temp = ""
    repeat_nline = 0
    # remove extra lines
    for i in str:
        if i == "\n":
            if repeat_nline <= 1:
                repeat_nline+=1
                temp += i
            else:
                continue
        else:
            if i != " ":
                repeat_nline = 0
            temp += i
            continue
    return temp

def str_token_count(str):
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(str))

class MyComment:
    def __init__(self, comment, pos, depth):
        self.comment = comment
        self.pos = pos
        self.depth = depth

### getting hot subreddit post

In [72]:
res = requests.get('https://oauth.reddit.com/r/environment/hot',
                   headers=headers)

In [79]:
res.json()['data']['children']

[{'kind': 't3',
  'data': {'approved_at_utc': None,
   'subreddit': 'environment',
   'selftext': '',
   'author_fullname': 't2_eaz2zp4x6',
   'saved': False,
   'mod_reason_title': None,
   'gilded': 0,
   'clicked': False,
   'title': "Animal-Free Whey Is Molecularly Identical to Cow's Whey but Uses 99% Less Water and Causes 97% Less Emissions",
   'link_flair_richtext': [],
   'subreddit_name_prefixed': 'r/environment',
   'hidden': False,
   'pwls': 6,
   'link_flair_css_class': None,
   'downs': 0,
   'thumbnail_height': 140,
   'top_awarded_type': None,
   'hide_score': False,
   'name': 't3_16cn08o',
   'quarantine': False,
   'link_flair_text_color': 'dark',
   'upvote_ratio': 0.98,
   'author_flair_background_color': None,
   'subreddit_type': 'public',
   'ups': 797,
   'total_awards_received': 0,
   'media_embed': {},
   'thumbnail_width': 140,
   'author_flair_template_id': None,
   'is_original_content': False,
   'user_reports': [],
   'secure_media': None,
   'is_reddit_

In [81]:
for post in res.json()['data']['children']:
    print(post['data']['title'])

Animal-Free Whey Is Molecularly Identical to Cow's Whey but Uses 99% Less Water and Causes 97% Less Emissions
Biden cancels Trump drilling leases in Alaska's largest wildlife refuge
Environmentalists condemn Australia’s ‘woeful record’ after 48 plants and animals added to threatened species list
Antarctica warming much faster than models predicted in ‘deeply concerning’ sign for sea levels
Antarctica warming much faster than models predicted in ‘deeply concerning’ sign for sea levels
‘Sleeping giant’ drought threatens more disasters after record Canada wildfires
Heat denial: influencers question validity of high temperatures
Climate protesters hit Ryanair boss with cream pies
More wild Atlantic salmon found in U.S. rivers than any time in the past decade, officials say
The Sierra Club hired its first Black leader. Turmoil over racial equity followed.
California is going big on offshore wind. That's good for climate, but not enough
Microplastics from tyres are polluting our waterways: s

getting results into pandas dataframe

In [28]:
import pandas as pd

In [62]:
df = pd.DataFrame()

In [82]:
for post in res.json()['data']['children']:
    df = pd.concat([df, 
                    pd.DataFrame({
                        'subreddit':[post['data']['subreddit']],
                        'title': [post['data']['title']],
                        'selftext': post['data']['selftext'],
                        'upvote_ratio': post['data']['upvote_ratio']              
                        })], 
                    axis=0,
                    ignore_index=True)
    df

### get post title

In [33]:
submission.title

"Scientists: Japan's Plan To Dump Nuclear Waste Into The Pacific Ocean May Not Be Safe"

### Scrape specified post (basic)

In [160]:
import praw
# Replace with your own credentials from your Reddit application
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=secret,
    user_agent='APP-NAME by REDDIT-USERNAME',
)
post_id = '16e3r4t'
# Get the Reddit submission (post) by its unique identifier
submission = reddit.submission(id=post_id)

Basic

In [161]:
# Retrieve and print all comments in the post

def scrape_comments_example(comment, depth=0):
    if depth==0:
        print("(first layer comment)")
    try:
        print('| ' * depth)
        print(add_indent(f'comment score: {comment.score}',depth))
        indented_comment = add_indent(f'Comment: {comment.body}', depth)
        print_str_as_blocks(indented_comment,indent=depth)\
        #print(f'Comment: {comment.body}')
        all_comments.append(comment.body)
        print("\n")
    except:
        return
    
    # Recursively scrape replies to this comment
    for reply in comment.replies:
        scrape_comments_example(reply, depth + 1)
    # end of first layer branch
    if depth==0:
        print("End of top comment branch")
        print("#"*100+"\n")
        
    # end of branch
    if len(comment.replies) == 0:
        print('-' * 30)

In [162]:
import tiktoken
# Print the post's title and other information
print("Post Title:", submission.title)
print("Author:", submission.author)
print("Score:", submission.score)
print("URL:", submission.url)
print("\n")

# Iterate through top-level comments and call the recursive function to scrape all comments
all_comments = []
for top_level_comment in submission.comments:
    scrape_comments_example(top_level_comment)

Post Title: I found something my ex-husband accused me of destroying, and I returned it.
Author: AlleyQV
Score: 10795
URL: https://www.reddit.com/r/pettyrevenge/comments/16e3r4t/i_found_something_my_exhusband_accused_me_of/


(first layer comment)

comment score: 4866
Comment: Getting in to that university is the most important thing that
ever happened to him.


| 
  comment score: 1743
  Comment: It's Harvard. Gotta be Harvard. MIT is the better school anyway.


| | 
    comment score: 1236
    Comment: “There's a chip on my shoulder
    And it's big as a boulder
    With the chance I've been given
    I'm gonna be driven as hell
    I'm so close, I can taste it
    So, I'm not gonna waste it
    Yeah, there's a chip on my shoulder
    You might wanna get one as well”
    
    It’s definitely Harvard. She referenced Legally Blonde the Musical.


| | | 
      comment score: 561
      Comment: I’ve never seen Legally Blonde the Musical, but when she
      said “they have a chip on their

In [163]:
str = ""
for comment in all_comments:
    str += comment
print("num comments: ", len(all_comments))
print("str len: ", len(str))
token_count = str_token_count(str)
print("token_count: ",token_count)

num comments:  486
str len:  90682
token_count:  20710


### Filtered scrape/targeted scrape

In [8]:
import praw
# Replace with your own credentials from your Reddit application
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=secret,
    user_agent='APP-NAME by REDDIT-USERNAME',
)
# japan nuclear waste post: ue6qnp
post_id = 'ue6qnp'
# Get the Reddit submission (post) by its unique identifier
submission = reddit.submission(id=post_id)

In [12]:
submission.comments[0].author

Redditor(name='BadChang')

In [6]:
'''
    description goes here
'''
terminate = False
total_token_count = 0
token_limit = 7000 # leave room for the ~8000 token limit
queue = []
queue_set_pointer = 1 # scrape first layer comments by sets of 10s
back_up_queue = []
scraped_comments = []
num_comment_layer = [10,10,3,3,3,3,10,10,10,10]
max_depth = 7
depth = 0

while not terminate:
    print(f"depth: {depth}, queue len: {len(queue)}")
    indent = "| "*depth
    next_queue = []

    if len(queue) == 0:
        if len(back_up_queue) > 0:
            print("#\n"*3+"."*10+f"scraping backup queue #{len(back_up_queue)}")
            queue = back_up_queue
            back_up_queue = []
        else:
            depth = 0
            indent = "| "*depth
            for i in range((queue_set_pointer-1)*num_comment_layer[0],
                           (queue_set_pointer)*num_comment_layer[0]):
                if i >= len(submission.comments):
                    terminate = True
                    break
                comment = MyComment(submission.comments[i], [i+1], depth)
                queue.append(comment)
            queue_set_pointer+=1

    for i, que in enumerate(queue):
        # scrape queued comments
        try:
            body = que.comment.body
        except:
            print("")
            continue

        this_comment = que.comment
        print(f"{indent} this comment pos: {que.pos}, total_token_before: {total_token_count}")  
        print(f"{indent} comment: {body[:20]}...")
        scraped_comments.append(que)
        token_count = str_token_count(this_comment.body)
        total_token_count+=token_count

        if total_token_count > token_limit:
            terminate = True
            break
        try:
            replies = this_comment.replies
        except:
            print("")
            continue
        if depth>=max_depth:
            print("")
            continue
        
        # add new comments to queue
        for j, reply in enumerate(this_comment.replies):
            print(f"{indent} reply # {j}", end="") 
            reply_pos = que.pos.copy()
            reply_pos.append(j+1)
            queued_comment = MyComment(reply, reply_pos, depth)
            if j+1 <= num_comment_layer[depth+1]:
                print(" *queued")
                next_queue.append(queued_comment)
            else:
                print("")
                back_up_queue.append(queued_comment)
        print("")

    queue = next_queue
    depth+=1
    


depth: 0, queue len: 0
 this comment pos: [1], total_token_before: 0
 comment: I feel like this bel...
 reply # 0 *queued
 reply # 1 *queued
 reply # 2 *queued
 reply # 3 *queued
 reply # 4 *queued
 reply # 5 *queued

 this comment pos: [2], total_token_before: 11
 comment: Have any of you actu...
 reply # 0 *queued
 reply # 1 *queued
 reply # 2 *queued
 reply # 3 *queued
 reply # 4 *queued
 reply # 5 *queued
 reply # 6 *queued

 this comment pos: [3], total_token_before: 83
 comment: Ok let’s talk about ...
 reply # 0 *queued
 reply # 1 *queued
 reply # 2 *queued
 reply # 3 *queued
 reply # 4 *queued
 reply # 5 *queued
 reply # 6 *queued
 reply # 7 *queued

 this comment pos: [4], total_token_before: 559
 comment: It isn't nuclear was...
 reply # 0 *queued
 reply # 1 *queued

 this comment pos: [5], total_token_before: 666
 comment: Do you want Godzilla...
 reply # 0 *queued
 reply # 1 *queued
 reply # 2 *queued
 reply # 3 *queued
 reply # 4 *queued
 reply # 5 *queued

 this comment p

### Grouping scrapped comments

this is ungroupped scrapped comments. Note: len(scraped_comments)

In [227]:
for comment in scraped_comments:
    print(comment.pos)
len(scraped_comments)

[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[2, 1]
[2, 2]
[2, 3]
[2, 4]
[2, 5]
[2, 6]
[2, 7]
[3, 1]
[3, 2]
[3, 3]
[3, 4]
[3, 5]
[3, 6]
[3, 7]
[3, 8]
[4, 1]
[4, 2]
[5, 1]
[5, 2]
[5, 3]
[5, 4]
[5, 5]
[5, 6]
[6, 1]
[6, 2]
[6, 3]
[6, 4]
[7, 1]
[7, 2]
[7, 3]
[7, 4]
[7, 5]
[7, 6]
[8, 1]
[8, 2]
[9, 1]
[10, 1]
[10, 2]
[10, 3]
[1, 2, 1]
[1, 2, 2]
[1, 2, 3]
[1, 3, 1]
[2, 2, 1]
[2, 2, 2]
[2, 2, 3]
[3, 1, 1]
[3, 1, 2]
[3, 2, 1]
[3, 2, 2]
[3, 4, 1]
[3, 6, 1]
[3, 7, 1]
[3, 7, 2]
[3, 8, 1]
[3, 8, 2]
[3, 8, 3]
[4, 1, 1]
[4, 1, 2]
[4, 2, 1]
[5, 3, 1]
[5, 4, 1]
[6, 1, 1]
[6, 1, 2]
[7, 2, 1]
[7, 4, 1]
[7, 5, 1]
[7, 5, 2]
[8, 1, 1]
[8, 2, 1]
[9, 1, 1]
[9, 1, 2]
[10, 1, 1]
[1, 2, 1, 1]
[1, 2, 2, 1]
[2, 2, 1, 1]
[2, 2, 1, 2]
[2, 2, 2, 1]
[3, 2, 1, 1]
[3, 4, 1, 1]
[3, 6, 1, 1]
[3, 7, 1, 1]
[3, 7, 1, 2]
[4, 1, 2, 1]
[4, 1, 2, 2]
[5, 4, 1, 1]
[6, 1, 2, 1]
[7, 2, 1, 1]
[7, 4, 1, 1]
[8, 1, 1, 1]
[8, 2, 1, 1]
[9, 1, 1, 1]
[10, 1, 1, 1]
[1, 2, 2, 1, 1]
[2, 2, 1, 1, 1]
[3, 4, 

177

In [27]:
class Discussion:
        def __init__(self, grouped_comments):
            self.grouped_comments = grouped_comments
            self.author = grouped_comments[0].comment.author

In [28]:
def group_comment_pos(group, depth=0):
    discussions = []
    seen_discussion = []
    print(f"\n\ndepth: {depth}")
    indent = "| "*depth
    this_group = []
    added_comments = [] # ones no need to sort anymore
    for comment in group:
        this_group = []
        if depth >= len(comment.pos):
            print("...... adding comment to discussion, depth > len(comment.pos)")
            print(f"{indent}added comment pos: {comment.pos}")
            added_comments.append(comment)
            discussions.append(comment)
            continue

        discussion_num = comment.pos[depth]
        if discussion_num in seen_discussion:
            continue

        print(f"\n{indent}####searching for group #{discussion_num} pos: {comment.pos}")
        for comment in group:
            if depth >= len(comment.pos):
                continue
            if comment.pos[depth] == discussion_num:
                print(f"{indent}appending discussion: {comment.pos}")
                this_group.append(comment)
        
        print("this group:")
        for skipped_comment in added_comments:
            print(skipped_comment.pos, end="")
        for comment in this_group:
            print(comment.pos, end=" ")
        if depth == 0:
            grouped_comments = group_comment_pos(this_group, depth=depth+1)
            discussion = Discussion(grouped_comments)
            discussions.append(discussion)
        else:
            discussions.extend(group_comment_pos(this_group, depth=depth+1))
        seen_discussion.append(discussion_num) 
        print("discussion len ", len(discussions))

        # force break
        if depth == 0:
            print("*"*20+"finished a first layer comment")
            # break
    
    print("*"*20+f"\nend of discussion branch {depth}\
          discussion num #{len(discussions)}")
    if len(discussions) == 0:
        for comment in group:
            print("this_group ", comment.pos)
        return group
    
    return discussions
    

In [29]:
# get groupped comments
discussions = group_comment_pos(scraped_comments)



depth: 0

####searching for group #1 pos: [1]
appending discussion: [1]
appending discussion: [1, 1]
appending discussion: [1, 2]
appending discussion: [1, 3]
appending discussion: [1, 4]
appending discussion: [1, 5]
appending discussion: [1, 6]
appending discussion: [1, 2, 1]
appending discussion: [1, 2, 2]
appending discussion: [1, 2, 3]
appending discussion: [1, 3, 1]
appending discussion: [1, 2, 1, 1]
appending discussion: [1, 2, 2, 1]
appending discussion: [1, 2, 2, 1, 1]
appending discussion: [1, 2, 2, 1, 1, 1]
appending discussion: [1, 2, 4]
appending discussion: [1, 2, 5]
this group:
[1] [1, 1] [1, 2] [1, 3] [1, 4] [1, 5] [1, 6] [1, 2, 1] [1, 2, 2] [1, 2, 3] [1, 3, 1] [1, 2, 1, 1] [1, 2, 2, 1] [1, 2, 2, 1, 1] [1, 2, 2, 1, 1, 1] [1, 2, 4] [1, 2, 5] 

depth: 1
...... adding comment to discussion, depth > len(comment.pos)
| added comment pos: [1]

| ####searching for group #1 pos: [1, 1]
| appending discussion: [1, 1]
this group:
[1][1, 1] 

depth: 2
...... adding comment to dis

Groupped results 

[a,b,c] represents the location of a comment relative to it's parent comments

In [32]:
# check if length is same as ungroupped comments
print(f"discussion len: {len(discussions)}")
for discussion in discussions:
    print(f"new discussion---------------author: {discussion.author}")
    for comment in discussion.grouped_comments:
        print(comment.pos)

discussion len: 30
new discussion---------------author: BadChang
[1]
[1, 1]
[1, 2]
[1, 2, 1]
[1, 2, 1, 1]
[1, 2, 2]
[1, 2, 2, 1]
[1, 2, 2, 1, 1]
[1, 2, 2, 1, 1, 1]
[1, 2, 3]
[1, 2, 4]
[1, 2, 5]
[1, 3]
[1, 3, 1]
[1, 4]
[1, 5]
[1, 6]
new discussion---------------author: homolicorn
[2]
[2, 1]
[2, 2]
[2, 2, 1]
[2, 2, 1, 1]
[2, 2, 1, 1, 1]
[2, 2, 1, 2]
[2, 2, 2]
[2, 2, 2, 1]
[2, 2, 3]
[2, 3]
[2, 4]
[2, 5]
[2, 6]
[2, 7]
new discussion---------------author: mcstandy
[3]
[3, 1]
[3, 1, 1]
[3, 1, 2]
[3, 2]
[3, 2, 1]
[3, 2, 1, 1]
[3, 2, 2]
[3, 3]
[3, 4]
[3, 4, 1]
[3, 4, 1, 1]
[3, 4, 1, 1, 1]
[3, 5]
[3, 6]
[3, 6, 1]
[3, 6, 1, 1]
[3, 6, 1, 1, 1]
[3, 6, 1, 1, 1, 1]
[3, 6, 1, 1, 1, 1, 1]
[3, 6, 1, 1, 1, 1, 1, 1]
[3, 7]
[3, 7, 1]
[3, 7, 1, 1]
[3, 7, 1, 2]
[3, 7, 2]
[3, 8]
[3, 8, 1]
[3, 8, 2]
[3, 8, 3]
new discussion---------------author: SchopenhauersFeline1
[4]
[4, 1]
[4, 1, 1]
[4, 1, 2]
[4, 1, 2, 1]
[4, 1, 2, 2]
[4, 2]
[4, 2, 1]
new discussion---------------author: None
[5]
[5, 1]
[5, 2]
[5, 3]
[5, 

### Extracting text

In [232]:
discussion_blocks = []
all_content = ""
for i, group in enumerate(discussions):
    block_of_comments = ""
    for comment in group:
        comment_str=comment.comment.body
        block_of_comments+=remove_extra_lines("{"+comment_str)+"}\n\n"
    seperator_a = f"**Begining discussion {i+1}\n"
    seperator_b = f"**End of discussion {i+1}\n\n"
    all_content+=seperator_a+block_of_comments+seperator_b
    discussion_blocks.append(block_of_comments)

In [233]:
# see groups of first layer and their discussions
print_str_as_blocks(discussion_blocks[0])

{I feel like this belongs in r/nottheonion ...}

{I assumed that’s where it was when I clicked on it tbh.}

{People get exposed to background radiations levels every day. Water is
good at absorbing radiation.

The people of Japan get a lot more radiation from all of the coal they burn.
They stupidly decided to switch to more coal after Fukushima.

Some day people will look back and shake their heads at how stupid everyone
was.}

{If we make it that far}

{well im doing it right now}

{It's tritium. That means it'll go into the water cycle and the food chain,
so no, water is not good at "absorbing" it at all, whatever that is supposed
to mean. Also, releasing fly ash from coal plants into the atmosphere is
not something that happens in developed countries any more.}

{At the concentrations that tritium will quickly achieve in the ocean, this
is a no nevermind.}

{Except that it will be concentrated within the food chain. And we're taking
about a whole load of tritium here.}

{No, becaus

In [234]:
print(f"all_content str len: {len(all_content)}")
print(f"all_content token count: {str_token_count(all_content)}")

all_content str len: 33506
all_content token count: 7771


In [235]:
print(all_content)

**Begining discussion 1
{I feel like this belongs in r/nottheonion ...}

{I assumed that’s where it was when I clicked on it tbh.}

{People get exposed to background radiations levels every day. Water is good at absorbing radiation.

The people of Japan get a lot more radiation from all of the coal they burn. They stupidly decided to switch to more coal after Fukushima.

Some day people will look back and shake their heads at how stupid everyone was.}

{If we make it that far}

{well im doing it right now}

{It's tritium. That means it'll go into the water cycle and the food chain, so no, water is not good at "absorbing" it at all, whatever that is supposed to mean. Also, releasing fly ash from coal plants into the atmosphere is not something that happens in developed countries any more.}

{At the concentrations that tritium will quickly achieve in the ocean, this is a no nevermind.}

{Except that it will be concentrated within the food chain. And we're taking about a whole load of tri

save to pikle file

In [236]:
import pickle
with open('./rsc/reddit-post-filtered-comments-ue6qnp.pkl','wb') as f:
    pickle.dump(all_content, f)